<a href="https://colab.research.google.com/github/PoiZeX/CloudCourse/blob/main/ex6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
import time
from ipywidgets.widgets.widget_style import Style
from zmq.constants import NULL
from firebase import firebase
import IPython
from google.colab import output
import random
import ipywidgets as widgets
from ipywidgets import VBox, HBox

questionsFromDB = []

def get_all_database():
  global questionsFromDB
  FBconn = firebase.FirebaseApplication('https://ex6game-default-rtdb.europe-west1.firebasedatabase.app/',None)
  res = FBconn.get('/Questions/',None)

  if res:
    for key in res:
      if key is not None: 
        questionsFromDB.append(key)

currentSelectedAns = ""
finalSelectedBtn = widgets.Button(description="") # null object
question = ""
randomQuestionAndAnswer = NULL
answers = []
buttonsLst = []

# Define a function to save the selected answer

def get_question_and_answers_dictionary():
        global randomQuestionAndAnswer, answers, question, radioButtonLst
        # Use display to add items which will be persisted on notebook reload.
        if (questionsFromDB):
          while randomQuestionAndAnswer == NULL:
            randomQuestionAndAnswer = random.choice(questionsFromDB)
        else:
          return

        if randomQuestionAndAnswer is not None:
          question = str(randomQuestionAndAnswer['question'])
        answers = [str(ans) for ans in randomQuestionAndAnswer['answers']]
        answers = answers[1:]

# on_click of answer button
def save_answer(b):
  global currentSelectedAns, finalSelectedBtn
  currentSelectedAns = str(b.description)  # save the answer to compare
  finalSelectedBtn = b  # save the button object to mark it later

def create_and_show_elements():
  global randomQuestionAndAnswer, answers, question, radioButtonLst
  display(IPython.display.HTML(f"<h2>The question is: {question}</h2>"))

  buttonsLst = [widgets.Button(description = answer, layout = widgets.Layout( width='auto', height='auto')  ) for answer in answers]

  for btn in buttonsLst:
    btn.on_click(save_answer)
    btn.style.button_color = 'orange'

  left_box = VBox([buttonsLst[0], buttonsLst[1]])
  right_box = VBox([buttonsLst[2], buttonsLst[3]])
  display(HBox([left_box, right_box]))


def on_submit_button_clicked(b):
    global finalSelectedBtn
    # iterate over question widgets and get user answers
    answerId = randomQuestionAndAnswer['correctAnswer']
    if answerId:
      correctAns = answers[int(answerId)-1]

    if currentSelectedAns is NULL:
      print("Please select answer")
      return

    #print(f"Question : {question}")
    #print(f"Correct answer: {correctAns}")
    #print(f"Selected answer: {currentSelectedAns}")
        
    # check if user's answer is correct
    if currentSelectedAns.strip() == correctAns.strip():
        print("Correct!")
        finalSelectedBtn.style.button_color = 'lightgreen'
        # start timer and refresh
        time.sleep(5)
        generate_new_question()
    else:
        finalSelectedBtn.style.button_color = 'red'
        display(IPython.display.HTML('<br/><h4>Incorrect answer<br/>Try Again! You can do it!<h4/><br/>'))



# refresh view -> get new question
def generate_new_question():
  output.clear()
  create_and_show_elements()
  display(submit_button)


##### MAIN #####
get_all_database()
get_question_and_answers_dictionary()
generate_new_question()

submit_button = widgets.Button(description="Submit") # prompt user to submit answers
submit_button.on_click(on_submit_button_clicked) # assign callback function to submit button

output.register_callback('notebook.onClicked', on_clicked)



Button(description='Submit', style=ButtonStyle())